In [ ]:
#hide
from nbdev.showdoc import show_doc

# 10-fold Cross Validation
> Code for training is the same as Notebook 05.

In [ ]:
from car_speech.fname_processing import load_fnames
from car_speech.pipeline import *

import string
from random import shuffle
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.model_selection import train_test_split, KFold

## Configuration

In [ ]:
DATASET_TYPE = 'digits' # 'digits' or 'letters' or 'mixed'
LEVEL_NAME = 'IDL' # 'IDL' or '35U' or '35D' or '55U' or '55D'

EPOCHS = 100

In [ ]:
if DATASET_TYPE == 'digits':
    label_strings = np.array([str(num) for num in range(0,10)])
    fname_str = 'digit'
elif DATASET_TYPE == 'letters':
    label_strings = np.array(list(string.ascii_uppercase))
    fname_str = 'letter'
elif DATASET_TYPE == 'mixed':
    label_strings = np.array([str(num) for num in range(0,10)] + list(string.ascii_uppercase))
    fname_str = 'mixed'
    
# load classified filenames
filenames = load_fnames('noise_levels/' + fname_str + '_noise_levels/' + LEVEL_NAME + '.data')
num_samples = len(filenames)
print('number of files:', len(filenames))

# shuffle
shuffle(filenames)

fold_no = 1
acc_per_fold = []

number of files: 1579


## Loop for 10-fold

In [ ]:
beg_ratio = 0
while beg_ratio < 0.9:
    test_beg = int(beg_ratio*num_samples)
    test_end = int((beg_ratio+0.1)*num_samples)
    
    fnames_copy = deepcopy(filenames)
    test_files = fnames_copy[test_beg:test_end]
    del fnames_copy[test_beg:test_end]
    
    val_len = len(test_files)
    val_files = fnames_copy[-val_len:]
    del fnames_copy[-val_len:]
    
    train_files = fnames_copy
    
    beg_ratio += 0.1
    
    train_files = shuffle_data(train_files)
    val_files = shuffle_data(val_files)
    test_files = shuffle_data(test_files)
    

    # Process data using the combined pipeline
    spectrogram_ds = preprocess_dataset(train_files, DATASET_TYPE)
    train_ds = spectrogram_ds
    val_ds = preprocess_dataset(val_files, DATASET_TYPE)
    test_ds = preprocess_dataset(test_files, DATASET_TYPE)
    print("Pipeline Completed")
    
    # batch
    batch_size = 64
    train_ds = train_ds.batch(batch_size)
    val_ds = val_ds.batch(batch_size)
    
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    train_ds = train_ds.cache().prefetch(AUTOTUNE)
    val_ds = val_ds.cache().prefetch(AUTOTUNE)
    
    # model
    for spectrogram, _ in spectrogram_ds.take(1):
        input_shape = spectrogram.shape
#     print('Input shape:', input_shape)
    num_labels = len(label_strings)

    norm_layer = preprocessing.Normalization()
    norm_layer.adapt(spectrogram_ds.map(lambda x, _: x))

#     model = models.Sequential([
#         layers.Input(shape=input_shape),
#         preprocessing.Resizing(32, 32), 
#         norm_layer,
#         layers.Conv2D(32, 3, activation='relu'),
#         layers.Conv2D(64, 3, activation='relu'),
#         layers.MaxPooling2D(),
#         layers.Dropout(0.25),
#         layers.Flatten(),
#         layers.Dense(128, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(num_labels),
#     ])

    model = models.Sequential([
        layers.Input(shape=input_shape),
        preprocessing.Resizing(32, 32), 
        norm_layer,
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_labels),
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )
    
    # Train
    history = model.fit(
        train_ds, 
        validation_data=val_ds,  
        epochs=EPOCHS,
        callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
    )
    
    # Test
    test_audio = []
    test_labels = []

    for audio, label in test_ds:
        test_audio.append(audio.numpy())
        test_labels.append(label.numpy())

    test_audio = np.array(test_audio)
    test_labels = np.array(test_labels)
    
    y_pred = np.argmax(model.predict(test_audio), axis=1)
    y_true = test_labels

    test_acc = sum(y_pred == y_true) / len(y_true)
    print(f'Fold {fold_no} - Test set accuracy: {test_acc:.0%}')
    acc_per_fold.append(round(test_acc, 2))
    fold_no += 1

print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Pipeline Completed
Epoch 1/100
20/20 [==============================] - 3s 152ms/step - loss: 2.2803 - accuracy: 0.1399 - val_loss: 2.1643 - val_accuracy: 0.2484
Epoch 2/100
20/20 [==============================] - 2s 117ms/step - loss: 2.0911 - accuracy: 0.2704 - val_loss: 1.8543 - val_accuracy: 0.3949
Epoch 3/100
20/20 [==============================] - 2s 118ms/step - loss: 1.8991 - accuracy: 0.3391 - val_loss: 1.6914 - val_accuracy: 0.4650
Epoch 4/100
20/20 [==============================] - 2s 118ms/step - loss: 1.7020 - accuracy: 0.3992 - val_loss: 1.4192 - val_accuracy: 0.5287
Epoch 5/100
20/20 [==============================] - 2s 117ms/step - loss: 1.5736 - accuracy: 0.4261 - val_loss: 1.3367 - val_accuracy: 0.5159
Epoch 6/100
20/20 [==============================] - 2s 118ms/step - loss: 1.4691 - accuracy: 0.4711 - val_loss: 1.2528 - val_accuracy: 0.5669
Epoch 7/100
20/20 [==============================] - 2s 120ms/step - loss: 1.3650 - accuracy: 0.5130 - val_loss: 1.2152 - v

20/20 [==============================] - 2s 117ms/step - loss: 0.4069 - accuracy: 0.8511 - val_loss: 0.5186 - val_accuracy: 0.8354
Epoch 00035: early stopping
Fold 2 - Test set accuracy: 80%
Pipeline Completed
Epoch 1/100
20/20 [==============================] - 3s 152ms/step - loss: 2.2550 - accuracy: 0.1528 - val_loss: 2.0730 - val_accuracy: 0.2848
Epoch 2/100
20/20 [==============================] - 2s 118ms/step - loss: 2.0378 - accuracy: 0.2716 - val_loss: 1.8138 - val_accuracy: 0.4051
Epoch 3/100
20/20 [==============================] - 2s 116ms/step - loss: 1.8148 - accuracy: 0.3698 - val_loss: 1.6272 - val_accuracy: 0.4494
Epoch 4/100
20/20 [==============================] - 2s 116ms/step - loss: 1.7015 - accuracy: 0.3990 - val_loss: 1.5255 - val_accuracy: 0.4810
Epoch 5/100
20/20 [==============================] - 2s 118ms/step - loss: 1.5410 - accuracy: 0.4561 - val_loss: 1.3791 - val_accuracy: 0.4747
Epoch 6/100
20/20 [==============================] - 2s 116ms/step - loss: 

20/20 [==============================] - 3s 125ms/step - loss: 0.5359 - accuracy: 0.8147 - val_loss: 0.6775 - val_accuracy: 0.8038
Epoch 00026: early stopping
Fold 4 - Test set accuracy: 78%
Pipeline Completed
Epoch 1/100
20/20 [==============================] - 4s 225ms/step - loss: 2.2890 - accuracy: 0.1330 - val_loss: 2.1687 - val_accuracy: 0.2342
Epoch 2/100
20/20 [==============================] - 3s 144ms/step - loss: 2.1217 - accuracy: 0.2272 - val_loss: 1.8278 - val_accuracy: 0.3671
Epoch 3/100
20/20 [==============================] - 3s 144ms/step - loss: 1.8386 - accuracy: 0.3365 - val_loss: 1.6445 - val_accuracy: 0.4620
Epoch 4/100
20/20 [==============================] - 3s 139ms/step - loss: 1.6806 - accuracy: 0.3721 - val_loss: 1.4846 - val_accuracy: 0.5063
Epoch 5/100
20/20 [==============================] - 3s 132ms/step - loss: 1.5435 - accuracy: 0.4466 - val_loss: 1.3730 - val_accuracy: 0.5316
Epoch 6/100
20/20 [==============================] - 3s 145ms/step - loss: 

20/20 [==============================] - 3s 126ms/step - loss: 1.4206 - accuracy: 0.4917 - val_loss: 1.1505 - val_accuracy: 0.6582
Epoch 7/100
20/20 [==============================] - 3s 125ms/step - loss: 1.3132 - accuracy: 0.5067 - val_loss: 1.1402 - val_accuracy: 0.6329
Epoch 8/100
20/20 [==============================] - 3s 125ms/step - loss: 1.2541 - accuracy: 0.5408 - val_loss: 1.0591 - val_accuracy: 0.6266
Epoch 9/100
20/20 [==============================] - 2s 125ms/step - loss: 1.1688 - accuracy: 0.5827 - val_loss: 1.0191 - val_accuracy: 0.6582
Epoch 10/100
20/20 [==============================] - 3s 128ms/step - loss: 1.1239 - accuracy: 0.5962 - val_loss: 1.0732 - val_accuracy: 0.5949
Epoch 11/100
20/20 [==============================] - 2s 124ms/step - loss: 1.0783 - accuracy: 0.6168 - val_loss: 0.9450 - val_accuracy: 0.6772
Epoch 12/100
20/20 [==============================] - 2s 125ms/step - loss: 1.0155 - accuracy: 0.6421 - val_loss: 0.8566 - val_accuracy: 0.7405
Epoch 13

20/20 [==============================] - 2s 124ms/step - loss: 1.2830 - accuracy: 0.5289 - val_loss: 1.1225 - val_accuracy: 0.6329
Epoch 8/100
20/20 [==============================] - 3s 125ms/step - loss: 1.2604 - accuracy: 0.5455 - val_loss: 1.0775 - val_accuracy: 0.6392
Epoch 9/100
20/20 [==============================] - 2s 124ms/step - loss: 1.1820 - accuracy: 0.5685 - val_loss: 0.9752 - val_accuracy: 0.6709
Epoch 10/100
20/20 [==============================] - 3s 129ms/step - loss: 1.1163 - accuracy: 0.5986 - val_loss: 0.9153 - val_accuracy: 0.6899
Epoch 11/100
20/20 [==============================] - 2s 124ms/step - loss: 1.0335 - accuracy: 0.6318 - val_loss: 0.8370 - val_accuracy: 0.6709
Epoch 12/100
20/20 [==============================] - 2s 124ms/step - loss: 0.9670 - accuracy: 0.6500 - val_loss: 0.8087 - val_accuracy: 0.7025
Epoch 13/100
20/20 [==============================] - 3s 126ms/step - loss: 0.9261 - accuracy: 0.6698 - val_loss: 0.8062 - val_accuracy: 0.7152
Epoch 1

In [ ]:
print("Done")
print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Done
[0.85, 0.8, 0.81, 0.78, 0.84, 0.85, 0.83, 0.77, 0.8, 0.87]
mean score: 0.82
